## Import libraries

In [1]:
import psycopg2

## Create a connection to the database and get a cursor

In [2]:
try: 
    conn = psycopg2.connect("host=localhost dbname=udacity_db user=root password=root")
    conn.autocommit = True
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
try: 
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get curser to the Database")
    print(e)

## Music store database with star schema

Imagine you work at an online Music Store. There will be many tables in our database, but let's just focus on 4 tables around customer purchases.  

![music_store_db_star_schema](music_store_db_star_schema.png)

### Fact table

In [3]:
# Create table
create_customer_transactions = """
CREATE TABLE IF NOT EXISTS customer_transactions (
    customer_id INT,
    store_id INT,
    spent NUMERIC,
    PRIMARY KEY (customer_id, store_id)
);
"""

try:
    cur.execute(create_customer_transactions)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

    
# Insert data into the table
insert_customer_transactions = """
INSERT INTO customer_transactions (customer_id, store_id, spent)
VALUES (%s, %s, %s)
"""

try:
    cur.executemany(insert_customer_transactions, ((1, 1, 20.50), (2, 1, 35.21)))
except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

## Dimesion tables

In [4]:
### customer

# Create table
create_customer = """
CREATE TABLE IF NOT EXISTS customer (
    customer_id INT PRIMARY KEY,
    name VARCHAR NOT NULL,
    rewards BOOLEAN
);
"""

try:
    cur.execute(create_customer)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

    
# Insert data into the table
insert_customer = """
INSERT INTO customer (customer_id, name, rewards)
VALUES (%s, %s, %s)
"""

try:
    cur.executemany(insert_customer, ((1, "Amanda", True), (2, "Toby", False)))
except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

In [5]:
### items_purchased

# Create table
create_items_purchased = """
CREATE TABLE IF NOT EXISTS items_purchased (
    customer_id INT PRIMARY KEY,
    item_number INT NOT NULL,
    item_name VARCHAR
);
"""

try:
    cur.execute(create_items_purchased)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

    
# Insert data into the table
insert_items_purchased = """
INSERT INTO items_purchased (customer_id, item_number, item_name)
VALUES (%s, %s, %s)
"""

try:
    cur.executemany(insert_items_purchased, ((1, 1, "Rubber Soul"), (2, 3, "Let It Be")))
except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

In [6]:
### store

# Create table
create_store = """
CREATE TABLE IF NOT EXISTS store (
    store_id INT PRIMARY KEY,
    state CHAR(2) NOT NULL
);
"""

try:
    cur.execute(create_store)
except psycopg2.Error as e:
    print("Error: Issue creating table")
    print(e)

    
# Insert data into the table
insert_store = """
INSERT INTO store (store_id, state)
VALUES (%s, %s)
"""

try:
    cur.executemany(insert_store, ((1, "CA"), (2, "WA")))
except psycopg2.Error as e:
    print("Error: Issue inserting rows")
    print(e)

## Queries

### Query 1
Find all the customers that spent more than 30 dollars, who are they, which store they bought it from, location of the store, what they bought and if they are a rewards member.

In [7]:
query = """
SELECT
    c.name,
    s.store_id,
    s.state,
    i.item_name,
    c.rewards
FROM
    customer_transactions t,
    customer c,
    items_purchased i,
    store s
WHERE
    t.customer_id = c.customer_id AND
    t.customer_id = i.customer_id AND
    t.store_id = s.store_id AND
    t.spent > 30;
"""

try:
    cur.execute(query)
except psycopg2.Error as e:
    print("Error while executing query")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

('Toby', 1, 'CA', 'Let It Be', False)


### Query 2
How much did Customer 2 spend?

In [8]:
query = """
SELECT
    customer_id,
    SUM(spent)
FROM
    customer_transactions
GROUP BY customer_id
HAVING customer_id = 2;
"""

try:
    cur.execute(query)
except psycopg2.Error as e:
    print("Error while executing query")
    print(e)

row = cur.fetchone()
while row:
    print(row)
    row = cur.fetchone()

(2, Decimal('35.21'))


## Drop the tables 

In [9]:
try:
    cur.execute("DROP TABLE customer_transactions, customer, items_purchased, store")
except psycopg2.Error as e:
    print("Error while dropping table")
    print(e)

## Close cursor and connection

In [10]:
cur.close()
conn.close()